In [1]:

import sys
sys.path.append("/Users/maryamsaad/Documents/ASR")

In [19]:
import logging
import sys
from module.src.pipeline import TranscriptionService


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)



audio_file = "Egyptian_Arabic_Conversational_Speech_Corpus/WAV/A0315_S006_0_G0349.wav"
    
service = TranscriptionService()
    
try:
    result = service.process_file(audio_file)
        
    print("\n--- Final Result ---")
    print(f"Raw: {result.full_raw_text[:100]}...")
    print(f"Corrected: {result.full_corrected_text[:100]}...")
        

    reviews = [s for s in result.segments if s.needs_review]
    if reviews:
        print(f"\nWARNING: {len(reviews)} segments need human review.")
    

            
except FileNotFoundError:
    logging.error("Audio file not found.")
except Exception as e:
    logging.error(f"Fatal error: {e}")



2025-11-29 00:46:35,151 - asr_pipeline - INFO - Initializing Transcription Service components...
2025-11-29 00:46:35,151 - asr_engine - INFO - Loading ASR model: omarxadel/hubert-large-arabic-egyptian on device mps
2025-11-29 00:46:38,202 - asr_pipeline - INFO - Starting processing for file: Egyptian_Arabic_Conversational_Speech_Corpus/WAV/A0315_S006_0_G0349.wav
2025-11-29 00:46:38,202 - root - ERROR - Fatal error: Error opening 'Egyptian_Arabic_Conversational_Speech_Corpus/WAV/A0315_S006_0_G0349.wav': System error.


In [20]:
output=result.full_corrected_text


In [21]:
import re

def add_newlines(text):
    # Add newline after . or ? if not already followed by one
    text = re.sub(r'([.?])\s*', r'\1\n', text)
    return text.strip()
corrected = result.full_corrected_text   # or whatever your field is
formatted = add_newlines(corrected)
print(formatted.replace(".",""))


طب خير النهارده يوم مميز جدا وكمان موضوعنا مميز وكمان ضيفتنا مميزة
أولاً خلينا نرحب بضيفتنا الآ غنية عن التعريف ونقول لها إن احنا النهارده حابين نشارك معها موضوع مهم جداً و شاركت معاها في موضوع مهم جدا، وهو موضوع حيستفيد منه كل حد حيسمعنا، كل فرد من أفراد الأسرة
أهلاً بك
في البداية كنت حابة اتكلم معاك النهاردة عن موضوعنا وهو عن الرياضة
حابين نعرف منك أهمية الرياضة وإني إزاي بتأثر على الإنسان وإزاي بتشكل فرق عظيم
ازاي بتشكل فارق عظيم في حياته؟ طبعاً احنا في غنى تعريف الرياضة، لأن الرياضة هي شيء أساسي لازم في حياة كل فرد من أفراد الأسرة من أصغر فرد لأكبر فرد
كثير مننا بيبقى مثلاً شايف إن الرياضة ليها سن معين، ولكن الرياضة هي ليها سن معين، ولكن الرياضة للأسف هي مش محدودة بسن خالص
كتير مننا بيقدر يحقق فيها إنجازات كبيرة جدًا سواء كان صغير جدًا في السن أو كبير جدًا في السن
فتعليق حضرتك إيه دكتورة؟ إيه أولًا كان خليني أقول لك إيه الشبكة بتقطع، لو سمحت
أنا مش سامعاك
شِت كده واضح
نعم، خليني أقول لك إن [" صحيح جدا، إحنا كلنا عارفين فعلاً إن الرياضة هي الأساس
الرياضة هي بتبتجل، اه، بتبتلنا عندنا

In [22]:
from jiwer import wer, cer
import os
import json
import numpy as np



def load_transcript(path):
    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    texts = []
    for line in lines:
        if not line.strip():
            continue

        # transcripts are like:
        # [0.170,1.120]   G0348   female    الو السلام عليكم
        parts = line.split("\t")
        text = parts[-1].strip()

        # skip tags like [NPS], [ENS]
        if text.startswith("[") and text.endswith("]"):
            continue

        texts.append(text)

    # merge all segments into one reference
    return " ".join(texts)

gt=load_transcript("/Users/maryamsaad/Documents/ASR/Egyptian_Arabic_Conversational_Speech_Corpus/TXT/A0315_S006_0_G0349.txt")
print("Ground Truth:", gt)


Ground Truth: قلها أزيك يا طنط أسراء الحمد لله كويس الحمد لله لسا من شوية كنت راجعة من النادي أيه بمارس شوية تمارين كرة وسباحة وشوية تمارين كذا على شان الدورة الدموية لأن الرياضة مهمة جداً للصحة بحب ألعب كرة وبحب أركب الخيل وبحب الرماية سواء بقى بالقوس او بالبندقية آه بحب الجري ولازم أساساً الأنسان كل شوية يكون قصدي كل يوم يجري ولو نص ساعة بس لأن الجري بنشط الدورة الدموية أما بالنسبة لبقية أنواع الرياضة زي كرة السلة والتنس ده كله أنا بصراحة مش بهتم بيه لا في رياضات ثانية بحبها زيها زي يعني ركوب الخيل الرماية كذا زي ما أنا قلت آه هو وقعته جامدة فعلاً بس هو ركوبه مش صعب يعني أنا ركبت قبل كذا هو العكس الخيل لو أنتِ خفتي منه هو هيحس بحاجة يعني هو هيحس أن خايفة وهيجري وهو كمان هيترعب فهيجري أه عندك مثلاً الخيل للهنود الحمر دول ده خيل يعني عدائي جداً بطريقة أنه بيقتل بشر عادي يعني يعني بحب أبو تريكو ومحمد صلاح واللي همَ اللاعبين المشهورين يعني دول أكثرهم يعني ممكن شوية أهتم بأخبار كريستيانو ورنالدو أو ميسي هو كان في مدريد وده الوقت في يوفنتوس هو من برشلونة من زمان ولسا ما طلعش بس أحتمال يطلع

In [23]:
import jiwer
from pyarabic.araby import strip_tashkeel, strip_tatweel, normalize_hamza

def normalize_arabic(text):
    text = text.replace("أ","ا").replace("إ","ا").replace("آ","ا")
    text = strip_tashkeel(text)
    text = strip_tatweel(text)
    text = normalize_hamza(text)
    return text

gt = normalize_arabic(gt)
output = normalize_arabic(output)
wer_value = jiwer.wer(gt, output)
cer_value = jiwer.cer(gt, output)
print(f"WER: {wer_value:.2%}")
print(f"CER: {cer_value:.2%}")

WER: 97.20%
CER: 73.75%
